In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

#### 层解读
输入2，输出 4
其中的一个样本：[x1, x2]

层计算的输出过程
xw + b = y （这里面 x，w，b，y 都是向量）

w11x1 + w21x2 + b1 = y1
w12x1 + w22x2 + b2 = y2
w13x1 + w23x2 + b3 = y3
w14x1 + w24x2 + b4 = y4

参数矩阵 2 * 4
w = 
w11  w12  w13  w14
w21  w22  w23  w24

x = 【x1  x2】

样本数是 2，输出是 2 * 4

In [3]:
x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.0481258  -0.03530071  0.06538339  0.07707448]
 [ 0.0481258  -0.03530071  0.06538339  0.07707448]], shape=(2, 4), dtype=float32)


In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)